Проанализируем текст с помощью udpipe.

In [5]:
! ./udpipe --input horizontal --output conllu --tokenize --tag --parse russian-syntagrus-ud-2.4-190531.udpipe < testset2.txt > parsed.conllu

Loading UDPipe model: done.


In [7]:
from conllu import parse

with open("parsed.conllu", "r") as fd:
    parsed_text = fd.read()
    
conllu_text = parse(parsed_text)

In [13]:
conllu_text[0][1]

OrderedDict([('id', 2),
             ('form', 'Я'),
             ('lemma', 'я'),
             ('upostag', 'PRON'),
             ('xpostag', None),
             ('feats',
              OrderedDict([('Case', 'Nom'),
                           ('Number', 'Sing'),
                           ('Person', '1')])),
             ('head', 3),
             ('deprel', 'nsubj'),
             ('deps', None),
             ('misc', None)])

Составим частотный список глаголов:

In [36]:
freq_verbs = {}

for sentence in conllu_text:
    for word in sentence:
        if word["upostag"] == "VERB":
            if word["lemma"] not in freq_verbs:
                freq_verbs[word["lemma"]] = 0 
            else:
                freq_verbs[word["lemma"]] += 1

In [38]:
for key in list(freq_verbs):
    if freq_verbs[key] < 51:
        freq_verbs.pop(key)

In [39]:
freq_verbs

{'заявить': 257,
 'подать': 273,
 'просить': 143,
 'признать': 344,
 'стать': 212,
 'сообщить': 216,
 'обратиться': 138,
 'отказаться': 116,
 'рассматривать': 98,
 'рассказать': 53,
 'рассмотреть': 89,
 'мочь': 348,
 'начаться': 73,
 'обвинить': 460,
 'решить': 85,
 'пытаться': 88,
 'использовать': 56,
 'напомнить': 96,
 'получить': 126,
 'передать': 67,
 'направить': 104,
 'удаться': 60,
 'сказать': 97,
 'иметь': 93,
 'делать': 76,
 'обжаловать': 101,
 'отказать': 58,
 'говорить': 138,
 'счесть': 54,
 'быть': 135,
 'оставить': 60,
 'дать': 69,
 'удовлетворить': 172,
 'вынести': 152,
 'требовать': 172,
 'являться': 112,
 'обвинять': 119,
 'принять': 150,
 'выплатить': 61,
 'назначить': 62,
 'постановить': 56,
 'обязать': 59,
 'принадлежать': 53,
 'подтвердить': 90,
 'находиться': 116,
 'арестовывать': 81,
 'доказать': 56,
 'предъявить': 96,
 'пояснить': 57,
 'согласиться': 61,
 'взыскать': 62,
 'отклонить': 68,
 'провести': 62,
 'приговорить': 122,
 'запретить': 55,
 'считать': 151,
 '

Выберем коллокации глагол + прямое дополнение. Я выбираю только ближайшего к глаголу соседа, поэтому мои коллокации несовершенны (в них могут попадать глагол + прилагательное в аккузативе), но я забила. Для существительного я не брала лемму, чтобы сохранить падежную связность с глаголом.

In [203]:
collocations = []

def conditions(word, next_word):
    cond = [
        next_word,
        word["upostag"] == "VERB",
        word["lemma"] in freq_verbs,
        next_word["feats"] and "Case" in next_word["feats"] and next_word["feats"]["Case"] == "Acc"
    ]
    return all(cond)

for sentence in conllu_text:
    for i, word in enumerate(sentence):
        if i < len(sentence) - 1 and conditions(word, sentence[i + 1]):
            collocations.append([word["lemma"], sentence[i + 1]["form"]])
        elif i < len(sentence) - 1 and conditions(sentence[i + 1], word):
            collocations.append([sentence[i + 1]["lemma"], word["form"]])

In [204]:
print(len(collocations))
collocations[:10]

2046


[['рассмотреть', 'вопрос'],
 ['начаться', 'понедельник'],
 ['обвинить', 'лубянское'],
 ['получить', 'сроки'],
 ['обвинить', 'олигарха'],
 ['обвинить', 'Юлию'],
 ['просить', 'суд'],
 ['рассматривать', 'ходатайство'],
 ['делать', 'вынужденный'],
 ['иметь', 'вид']]

Считаем log-likelihood, dice, PMI:

In [205]:
import nltk
from nltk.collocations import *

bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(collocations)

In [206]:
pmi = finder.nbest(bigram_measures.pmi, 100)
pmi[:10]

[('нет', 'судебного'),
 ('связать', 'подзащитного'),
 ('отказаться', 'Тегеран'),
 ('отказаться', 'Чечню'),
 ('сказать', 'Усачева'),
 ('сказать', 'что-то'),
 ('постановить', 'присяжные'),
 ('пояснить', 'суть'),
 ('говорить', 'друга'),
 ('говорить', 'что')]

In [207]:
lr = finder.nbest(bigram_measures.likelihood_ratio, 100)
lr[:10]

[('просить', 'суд'),
 ('подать', 'иск'),
 ('принять', 'решение'),
 ('удовлетворить', 'иск'),
 ('удовлетворить', 'ходатайство'),
 ('вынести', 'решение'),
 ('обжаловать', 'решение'),
 ('иметь', 'право'),
 ('вынести', 'приговор'),
 ('обвинить', 'его')]

In [208]:
scores = finder.score_ngrams(bigram_measures.dice)
dice = sorted([x for x in scores if x[1] != 1.0], key=lambda x: x[1], reverse=True)[:100]
dice = [x[0] for x in dice]
dice[:10]

[('отказаться', 'Тегеран'),
 ('отказаться', 'Чечню'),
 ('сказать', 'Усачева'),
 ('сказать', 'что-то'),
 ('просить', 'суд'),
 ('говорить', 'друга'),
 ('говорить', 'что'),
 ('заявить', 'отвод'),
 ('находиться', 'время'),
 ('постановить', 'присяжные')]

Найдем пересечение всех топов-100:

In [209]:
union = set(dice) & set(lr) & set(pmi)
union

{('взыскать', 'недостающую'),
 ('доказать', 'невиновность'),
 ('запретить', 'деятельность'),
 ('заявить', 'отвод'),
 ('заявить', 'целый'),
 ('мочь', 'список'),
 ('находиться', 'время'),
 ('объявить', 'перерыв'),
 ('объявить', 'предпринимателя'),
 ('принадлежать', 'МИАНу'),
 ('принадлежать', 'ЮКОСу'),
 ('решить', 'проблему'),
 ('сообщить', 'ответ'),
 ('счесть', 'Касьянова'),
 ('счесть', 'доводы')}

Сравним со словарем глагольной сочетаемости:

In [210]:
with open("verb_coll.txt", "r") as fd:
    dict_text = fd.read().split("\n")
    
dict_collocations1 = [(token.split()[-2], token.split()[-1]) for token in dict_text if len(token.split()) > 2]
dict_collocations2 = [(token.split()[-1], token.split()[-2]) for token in dict_text if len(token.split()) > 2]

In [212]:
new_union = union & set(dict_collocations1) & set(dict_collocations2)
new_union

{('решить', 'проблему')}

Получилось негусто. Полагаю, что это из-за моей халатности c поиском коллокаций. Я считаю, что в этот список почем зря не внесены следущие пары слов:

объявить перерыв

сообщить ответ

запретить деятельность

доказать невиновность

Я считаю их коллокациями, так как для перечисленных глаголов их существительные являются единственными внутренними аргументами (что не верено, например, для пары слов "счесть Касьянова" - тут явно не хватает еще одного аргумента в инструменталисе) и они семантически связаны в том числе и как устойчивые выражения. Наример, нормально - "объявить перерыв", а "сказать перерыв" - невозможно. Добавим их в объединение вручную.

In [217]:
ultimate_union = new_union | {("объявить", "перерыв"), ("сообщить", "ответ"),
                              ("запретить", "деятельность"), ("доказать", "невиновность")}
ultimate_union

{('доказать', 'невиновность'),
 ('запретить', 'деятельность'),
 ('объявить', 'перерыв'),
 ('решить', 'проблему'),
 ('сообщить', 'ответ')}

Не очень понимаю, как отранжировать эти коллокации руками, но допустим, что я буду считать коллокации тем более "коллокативной", чем более устойчивое выражение оно составляет (нельзя подставить другой глагол, например).

In [233]:
from scipy import stats

ranged_collocations = [['объявить', 'перерыв'],
                       ['доказать', 'невиновность'],
                       ['решить', 'проблему'],
                       ['запретить', 'деятельность'],
                       ['сообщить', 'ответ']]

stats.spearmanr(ranged_collocations)

SpearmanrResult(correlation=0.6, pvalue=0.28475697986529375)

In [234]:
scores = finder2.score_ngrams(bigram_measures.dice)
print(scores)
dice = [x[0] for x in scores]
print(stats.spearmanr(dice))
dice

[(('доказать', 'невиновность'), 1.0), (('запретить', 'деятельность'), 1.0), (('объявить', 'перерыв'), 1.0), (('решить', 'проблему'), 1.0), (('сообщить', 'ответ'), 1.0)]


[('доказать', 'невиновность'),
 ('запретить', 'деятельность'),
 ('объявить', 'перерыв'),
 ('решить', 'проблему'),
 ('сообщить', 'ответ')]

In [235]:
lr = finder2.nbest(bigram_measures.likelihood_ratio, 5)
print(stats.spearmanr(lr))
lr

SpearmanrResult(correlation=0.6, pvalue=0.28475697986529375)


[('доказать', 'невиновность'),
 ('запретить', 'деятельность'),
 ('объявить', 'перерыв'),
 ('решить', 'проблему'),
 ('сообщить', 'ответ')]

In [231]:
finder2 = BigramCollocationFinder.from_documents(ranged_collocations)
pmi = finder2.nbest(bigram_measures.pmi, 5)
print(stats.spearmanr(pmi))
pmi

SpearmanrResult(correlation=0.6, pvalue=0.28475697986529375)


[('доказать', 'невиновность'),
 ('запретить', 'деятельность'),
 ('объявить', 'перерыв'),
 ('решить', 'проблему'),
 ('сообщить', 'ответ')]